In [1]:
import numpy as np
import pandas as pd
import time

#-------global variable---------
N_STATES = 40  
Ｍ_STATES = 30
ACTIONS = ['left', 'right', 'up', 'down']    
EPSILON = 0.95   
ALPHA = 0.15     
GAMMA = 0.95    
MAX_EPISODES = 300   
FRESH_TIME = 0.001    


In [2]:
#--------Build Q table----------
def build_q_table(n_states, m_states, actions): 
    table = pd.DataFrame(np.zeros((n_states * m_states, len(actions))),
                         columns=actions,)   
    return table

#--------choose action-----------
def choose_action(state, q_table): 
    state_actions = q_table.iloc[state, :] 
    #act non-greedy or state-action have no value
    if (np.random.uniform() > EPSILON) or ((state_actions == 0).all()):  
        action_name = np.random.choice(ACTIONS) 
    else:   # act greedy
        action_name = state_actions.idxmax()
    return action_name


In [3]:
#--------Building feedback---------
def get_env_feedback(S, A): 
    n_now_states=int(S / N_STATES) 
    m_now_states=int(S % N_STATES) 
    #print(n_now_states,m_now_states)
    if A == 'right':    # move right
        if m_now_states == N_STATES - 2 and n_now_states== M_STATES-1:   # terminate
            S_ = 'terminal'
            R = 1
        elif (S+1) % N_STATES ==0 :
            S_ = S
            R = 0
        else:
            S_ = S + 1
            R = 0
    elif A == 'left':    # move left
        R = 0
        if S % N_STATES ==0 :
            S_ = S
            R = 0
        else:
            S_ = S - 1
            R = 0
    elif A == 'down':    # move down
        if  n_now_states == M_STATES - 2 and m_now_states== N_STATES-1:   # terminate
            S_ = 'terminal'
            R = 1
        elif S >= N_STATES*(M_STATES-1) :
            S_ = S
            R = 0
        else:
            S_ = S + N_STATES
            R = 0
    elif A == 'up':    # move up
        R = 0
        if S <= N_STATES :
            S_ = S
            R = 0
        else:
            S_ = S - N_STATES
            R = 0
    return S_, R


In [4]:

#-----------renew environment--------------
def update_env(S, episode, step_counter):
    # This is how environment be updated
    env_list = ['-']*(N_STATES*M_STATES-1) + ['T']   # '---------T' our environment
    if S == 'terminal':
        interaction = 'Episode %s: total_steps = %s' % (episode+1, step_counter)
        print('\r{}'.format(interaction))
        time.sleep(2)
        #print('\r                                ')
    else:
        env_list[S] = 'o'
        #print('\r{}'.format('******************'))
        for i in range(M_STATES):
            interaction = ''.join(env_list[N_STATES*i:N_STATES*(i+1)])    
            #print('\r{}'.format(interaction))
        time.sleep(FRESH_TIME)


In [5]:
#----------building reinforcement learning-----------
def rl():
    # main part of RL loop
    q_table = build_q_table(N_STATES, M_STATES, ACTIONS) 
    for episode in range(MAX_EPISODES):
        step_counter = 0
        S = 0
        is_terminated = False
        update_env(S, episode, step_counter)
        while not is_terminated:
 
            A = choose_action(S, q_table)
            S_, R = get_env_feedback(S, A)  # take action & get next state and reward
            q_predict = q_table.loc[S, A]
            if S_ != 'terminal':
                q_target = R + GAMMA * q_table.iloc[S_, :].max()   # next state is not terminal
            else:
                q_target = R     # next state is terminal
                is_terminated = True    # terminate this episode
 
            q_table.loc[S, A] += ALPHA * (q_target - q_predict)  # update
            S = S_  # move to next state
 
            update_env(S, episode, step_counter+1)
            step_counter += 1
    return q_table

if __name__ == "__main__":
    q_table = rl()
    print('\r\nQ-table:\n')
    print(q_table)

Episode 1: total_steps = 11210
Episode 2: total_steps = 11217
Episode 3: total_steps = 4799
Episode 4: total_steps = 3818
Episode 5: total_steps = 8431
Episode 6: total_steps = 9019
Episode 7: total_steps = 5210
Episode 8: total_steps = 2253
Episode 9: total_steps = 2118
Episode 10: total_steps = 1098
Episode 11: total_steps = 24173
Episode 12: total_steps = 2474
Episode 13: total_steps = 6471
Episode 14: total_steps = 11919
Episode 15: total_steps = 9159
Episode 16: total_steps = 10256
Episode 17: total_steps = 10224
Episode 18: total_steps = 4840
Episode 19: total_steps = 2556
Episode 20: total_steps = 1655
Episode 21: total_steps = 1241
Episode 22: total_steps = 1619
Episode 23: total_steps = 5023
Episode 24: total_steps = 3237
Episode 25: total_steps = 1840
Episode 26: total_steps = 2745
Episode 27: total_steps = 2458
Episode 28: total_steps = 754
Episode 29: total_steps = 2974
Episode 30: total_steps = 4907
Episode 31: total_steps = 1250
Episode 32: total_steps = 7711
Episode 33: 